# *中美贸易战*——国内外新闻分析
###    李顺立 房鸿宇 徐嘉 裴婷 曲咏晗
### 2019.05.14
----
----

## 中新网爬取中美贸易战新闻
+ 导入模块
+ 使用selenium打开浏览器
+ 定义根据链接查找新闻的函数（使用request）url\_get\_content(url)
+ 定义时间转化函数time\_trans(time\_need)
+ 定义中新网爬取函数china_news(pages=5)
+ 结果预览

## WashingtongPost网站爬取
* 定义WashingtonPost网站爬取函数Washington_Post_news(page=5)
* 数据处理
----

### 导入模块

In [1]:
import time
from selenium.webdriver.chrome.options import Options
import numpy as np
from selenium import webdriver
import pandas as pd
import re
import os
import requests                        #打开链接
from bs4 import BeautifulSoup          #解码链接
from urllib.parse import urlencode     #输入内容转码

from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
time.sleep(np.random.rand()+0.5)

----
### 使用selenium打开浏览器

In [2]:
##不打开浏览器
options = Options() 
options.add_argument("--headless") 


#打开浏览器
driver = webdriver.Chrome()   #options = options
driver.get('http://sou.chinanews.com/search.do')  #打开链接

driver.find_element_by_xpath('//*[@id="q"]').send_keys('中美贸易战') #输入关键词
driver.find_element_by_xpath('/html/body/div[2]/form/table/tbody/tr/td/table/tbody/tr[2]/td[1]/input[2]').click()   #点击搜索

----
### 定义根据链接查找新闻的函数（使用request）url_get_content(url)

In [72]:
def url_get_content(url):
    """
    该函数根据所传入的字符串类型的链接url获取该页的内容
    示例：url = 'https://baidu.com'
    结果：输出为预处理后的结果
    """
    content = ''
    try:
        html = requests.get(url) #, cookies = cookies, headers = headers) #打开链接  #伪装成真实的访问者
        soup = BeautifulSoup(html.content, "html.parser")  #解码链接
        content0 = soup.find('div', {"class": "left_zw"}).get_text() #解码后找到相应内容
        content1 = ','.join(re.findall(r'\w+', content0))  #处理初始数据
        
        #试图处理数据
        try:
            remove = re.findall(r'function.*?fix', content1)[0]  
            content = content1.replace(remove, '')
        except:
            content  = content1
            
    except:
        pass
    
    return content

In [74]:
url = 'http://www.chinanews.com/business/2019/04-26/8821443.shtml'
url_get_content(url)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'中新网4月26日电,今天,由创业黑马主办,全面战略合作伙伴汇桔网支持的,2019教育未来产业峰会,暨黑马教育产业大课,在京举行,创业黑马董事长,黑马学院院长牛文文出席活动,并以,教育产业如何重做,为主题发表了演讲,创业黑马董事长,黑马学院院长牛文文,牛文文在演讲中表示,大数据,人工智能等技术和基础设施的发展,为教育产业的升级提供了一个很好的基础,教育是有限用户,讲究精准,匹配,算法,算力等技术对此至关重要,此外,牛文文还提到,教育领域创业者在创业时一定不要忘记,社会公众价值大于用户价值,他指出,这些年来,创业黑马在企业家导师,投资人导师方面的积累非常雄厚,而且,至今为止,黑马社群中有几十家上市公司,比如尚德教育和51talk都在美国上市了,徐小平老师和俞敏洪老师也一直陪伴着创业黑马,此外,创业黑马和蓝象资本开设了教育产业实验室,和华图教育联合创始人于洪泽老师开设了教育产业实验室,今天还会发布几个教育产业实验室,希望能够与教育领域的上市公司,龙头企业,投资机构一起探讨行业未来,帮助创业者做成长加速辅导,创业黑马高级副总裁罗浛予,在峰会上,创业黑马高级副总裁罗浛予宣布黑马教育产业实验室正式发布,罗浛予表示,教育产业实验室带给创业者的是解析教育产业头部公司成长案例,让创业者学习教育产业独角兽成长的关键一招,实验室分为四个模块,一,搞定模式,搭建满足用户需求的创新模式,二,搞定产品,打造具备产品力与口碑的爆款产品,三,搞定团队,构建具有战斗力的团队与团队运营赋能,四,搞定资源,打通链接头部企业与资本市场的通道,目前,黑马教育产业实验室已入驻头部企业包括华图教育,VIPKID,瑞思英语,达内教育,开元股份,中大英才,腾讯课堂,iTutorGroup,高思教育,慧科集团,松鼠AI智适应教育,21世纪教育和用友新道等,黑马教育产业实验室具体以,商学院,孵化器模式,方式链接产业资源,拆解行业标杆案例,模拟企业发展路径,升级商业模式,对接产业资本,打造商业IP,在黑马教育产业实验室里,学员将获得教育产业最前沿的商业方法论,商业模式深度梳理与优化和产业上中下游资源及资本链接,2019教育未来产业峰会,暨黑马教育产业大课,在京举行,让中小创业者在巨头的赋能下产业升级,近年来,在互联网技术,消费升级,中美贸易战的宏观环境催化下,中国产业面临新一轮升级挑战,每个垂直产业中的上,中,下游企业

----
### 定义时间转化函数time_trans(time_need)

In [73]:
def time_trans(time_need):
    """
    将时间为s的数据转化为 x天xx:xx:xx的形式
    """
    hour = time_need // 3600 #整除,求小时数
    days = hour // 24 #求天数
    hours = hour % 24 #求小时数
    minutes = (time_need % 3600) // 60 #先取余数，再取整除，求分钟数
    seconds =  (time_need % 3600) % 60    #求秒数
    
    return days,hours,minutes,seconds  #返回时间

In [75]:
days,hours,minutes,seconds = time_trans(4600)
print('%d天 %02.0f: %02.0f: %02.0f'%(days,hours,minutes,seconds))

0天 01: 16: 40


----
## 定义中新网爬取函数china_news(pages=5)
[中新网搜索](http://sou.chinanews.com/search.do)

In [6]:
def china_news(pages = 5):
    """
    使用selemiun打开网址后，调用该函数，可以得到中新网的新闻内容。
    参数pages表示新闻内容的页数，默认为5页。
    返回值为包含【标题，链接，时间，内容】DataFrame格式的数据
    
    """


    import time as TM

    time_begin = TM.time()  #初始化时间
    col = 0  #查找的第i个内容
    page = 1    #初始化页数
    column_name = ['title', 'url', 'date_time', 'content']   #所获取的内容
    data = pd.DataFrame(columns=column_name)  #初始化结果
    
    print('-----'*8, '\n准备爬取第1页的内容！！')
    os.system('say "准备爬取第1页的内容!"')
    
    while page <= pages:  #页数还没爬完就继续
        
        key = True  #判断是否爬完了某一页的所有内容
        i = 1   #初始化当前页的新闻个数
        
        while key:  #爬取当前页的内容
            
            try:

                url = '//*[@id="news_list"]/table[%s]'%(i)   #第i个新闻的位置
                title_url = '/tbody/tr[1]/td[2]/ul/li[1]/a'   #第i个新闻的标题位置
                date_time_url = '/tbody/tr[2]/td/ul/li'   #第i个新闻的时间信息，包含了链接和时间


                title = driver.find_element_by_xpath(url+title_url).text  #标题
                href = driver.find_element_by_xpath(url+title_url).get_attribute('href')  #链接
                date_time0 = driver.find_element_by_xpath(url+date_time_url).text #时间+链接
                date_time = re.findall(r'\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d', date_time0)[0]  #只要时间
                content = url_get_content(href)  #获得链接中的内容

                #整理数据到data
                data.loc[col, :] = [title, href, date_time, content]

                i += 1  #更新到下一个新闻
                col += 1  #更新col index
                
                ##每10条数据休息一会并告知我
                if col % 10 == 0:
                    print('已经爬取数据%s条！'%(col))  
                    os.system('say "叮叮"')
                    os.system('say "%s"'%(col))
                    TM.sleep(np.random.rand()+3)   #每爬完一页的内容多休息一会
                
                TM.sleep(np.random.rand()+1)  #每爬完一个新闻暂停一会

            except:
                key = False
                
                ##估算剩余时间
                time_cost = TM.time() - time_begin  #当前所用时间
                print('已经用时%.2f秒!!'%(time_cost))
                time_mean = time_cost/page  #当前每页的平均时间        
                time_need = time_mean*(pages-page)   #剩余页数的时间
                days,hours,minutes,seconds = time_trans(time_need)  #单位为秒的时间数据处理为日期
                print('预计还需用时\"%d天%02.0f:%02.0f:%02.0f\"!'%(days, hours, minutes, seconds))
                print('-----'*4)
                
                page += 1   #更新到下一页
                if page <= pages:
                    print('正在爬取第%s页的内容！！'%(page))
                    os.system('say "正在爬取第%s页的内容"'%(page))
                    ##点击下一页
                    driver.find_element_by_link_text('下一页').click()   #点击下一页
                
    time_end = TM.time()-time_begin #计算总的时间
    days_end,hours_end,minutes_end,seconds_end = time_trans(time_end)
    
    print('用时%d天%02.0f:%02.0f:%02.0f，已经爬完%s页的内容共%s条！！'%( days_end,hours_end,minutes_end,seconds_end, page-1,col))  #已经爬完数据
    print('******'*8)
    
    driver.quit()  #退出并关闭浏览器
    
    os.system('say "中新网内容已经爬取完成！请查收！"')  #告知我
    
    return data
        
        
    

----
###  结果预览

In [7]:
data = china_news(53)  #爬取53页

---------------------------------------- 
准备爬取第1页的内容！！


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


已经爬取数据10条！
已经用时28.43秒!!
预计还需用时"0天00:24:38"!
--------------------
正在爬取第2页的内容！！


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


已经爬取数据20条！
已经用时59.90秒!!
预计还需用时"0天00:25:28"!
--------------------
正在爬取第3页的内容！！


...

已经爬取数据420条！
已经用时1316.04秒!!
预计还需用时"0天00:05:45"!
--------------------
正在爬取第43页的内容！！


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


已经爬取数据430条！
已经用时1345.97秒!!
预计还需用时"0天00:05:13"!
--------------------
正在爬取第44页的内容！！


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


已经爬取数据440条！
已经用时1378.38秒!!
预计还需用时"0天00:04:42"!
--------------------
正在爬取第45页的内容！！
已经爬取数据450条！
已经用时1406.08秒!!
预计还需用时"0天00:04:10"!
--------------------
正在爬取第46页的内容！！
已经爬取数据460条！
已经用时1436.11秒!!
预计还需用时"0天00:03:39"!
--------------------
正在爬取第47页的内容！！
已经爬取数据470条！
已经用时1462.89秒!!
预计还需用时"0天00:03:07"!
--------------------
正在爬取第48页的内容！！
已经爬取数据480条！
已经用时1491.69秒!!
预计还需用时"0天00:02:35"!
--------------------
正在爬取第49页的内容！！
已经爬取数据490条！
已经用时1521.74秒!!
预计还需用时"0天00:02:04"!
--------------------
正在爬取第50页的内容！！
已经爬取数据500条！
已经用时1554.22秒!!
预计还需用时"0天00:01:33"!
--------------------
正在爬取第51页的内容！！
已经爬取数据510条！
已经用时1585.12秒!!
预计还需用时"0天00:01:02"!
--------------------
正在爬取第52页的内容！！
已经爬取数据520条！
已经用时1614.28秒!!
预计还需用时"0天00:00:31"!
--------------------
正在爬取第53页的内容！！
已经用时1625.74秒!!
预计还需用时"0天00:00:00"!
--------------------
用时1625.74秒已经爬完53页的内容共524条！！
************************************************


In [52]:
data.head()

,title,url,date_time,content
0,磋商要平等互利 得适应打打谈谈,http://www.chinanews.com/gn/2019/05-11/8833614...,2019-05-11 07:02:36,"5月11日凌晨,第十一轮中美经贸磋商传出结束的消息,据说,在磋商结束后,刘鹤副总理接受了中国..."
1,国际锐评：解决中美经贸摩擦 关键在照顾彼此关切,http://www.chinanews.com/gn/2019/05-11/8833591...,2019-05-11 03:42:43,"国际锐评,解决中美经贸摩擦,关键在照顾彼此关切,第十一轮中美经贸高级别磋商当地时间星期五,1..."
2,关税大棒损人害己,http://www.chinanews.com/gn/2019/05-10/8833026...,2019-05-10 15:05:31,"国际锐评,关税大棒损人害己,华盛顿时间10日0点01分,北京时间10日12点01分,美方对2..."
3,浙大公布自主招生初审名单,http://www.chinanews.com/sh/2019/05-09/8831841...,2019-05-09 11:32:50,"计划招生不超过197人,一共有1534人通过初审,浙大公布自主招生初审名单学科竞赛省一等奖是..."
4,社评：美国要摆“鸿门宴”，但吓唬不住中国,http://www.chinanews.com/gn/2019/05-09/8831586...,2019-05-09 08:31:29,"美国,联邦公报,于北京时间8日晚上刊登一份公告称,从星期五开始,美国将把2000亿美元中国输..."


In [51]:
data.to_csv('中美贸易战2.csv', index = False)   #导出数据

---
###  数据处理

In [77]:
trade_war = pd.read_csv('中美贸易战2.csv')

In [78]:
trade_war.content[0]

'5月11日凌晨,第十一轮中美经贸磋商传出结束的消息,据说,在磋商结束后,刘鹤副总理接受了中国记者的集体采访,回答了磋商的有关问题,据陶然笔记了解,有关回答立场坚定,态度坦诚,大气从容,内容详实,还披露了关于近期磋商的一些情况,大致谈了下面几层意思,一是截至第十一轮磋商双方所取得的成果,,二是中方首度公开与美方三个分歧,三是清楚表明加征关税必然触发中方反制,四是表达了对中美关系前景的基本态度,同时透露双方同意继续保持沟通,最重要的消息,其实是第四条,因为美方提到继续增加关税,所以外界一直在猜测,中美磋商是不是破裂了,据了解,从答记者问的内容看,双方团队同意保持沟通,这也表明双方的磋商意愿并未彻底消失,与其说是破裂,不如可以看成是在双方分歧短期无法解决的情况下,磋商进程中发生的插曲,说完声明,再说几点感受,首先,中美一个主要分歧,恐怕还在于双方对,公平,的认知不同,就在这两天,美方频频通过外媒发声,将此轮磋商出现曲折,乃至于重新加征关税的原因,归咎于中方在磋商中,不遵守已谈成的承诺,这个说法经过层层放大,在海外很有市场,传到国内,有些朋友觉得听上去也是那么回事,其实只要稍加留意,就能注意到里面的两处破绽,不难看清这是美方为撇清磋商受阻的责任而设下的舆论议题,第一,磋商尚未完成,没有签署,严格来说,最终签署之前的任何变化都应看作是磋商过程之一,不遵守承诺,的提法其实无从谈起,长于各种谈判技法的美方,对此应该是心知肚明的,第二,美方说中方没有满足美方要求,所以是,不公平,的,但美方对中方诉求只字不提,这难道就公平了,看来在对,公平,的定义上,两边还存在分歧,中方认为相互尊重,平等互利才是公平的,而美方的叙述里,仿佛不满足自己的要求就是不公平的,怎么看这种分歧呢,可以看看一位长者曾对我提过的观点,两国磋商,如果谈出一个两边都不是很满意但又大致能够接受的结果,那就是成功的,执行起来也会顺利一些,如果只是单方面特别满意的协议,而无视另一方的利益和诉求,那么即便是签署了也未必能够落地,而且后患无穷,这也正像我们之前说的,如果某一方仅仅只考虑自己的诉求,以为可以通过极限施压来迫使另一方就范,而忽视了磋商的公平,那么也不是没有不欢而散的可能,其次,中美经贸磋商打打谈谈这种情况很有可能常态化,需要尽快去适应,第十一轮经贸磋商,就是个最典型的例子,磋商这边还谈着呢,那边美方就把2000亿

In [79]:
###加载自定义词典
jieba.load_userdict('my_dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_4/0d6f0wv952s2rc4jhfhtzstw0000gn/T/jieba.cache
Loading model cost 0.678 seconds.
Prefix dict has been built succesfully.


In [80]:
stop_words_null = ['2019', 'lex', 'nbsp', 'the', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ']
###中文停用词
with open('中文停用词.txt', 'r') as f:
    stop_words = stop_words_null
    for line in f:
        if '\ufeff' in line:
            word = re.split(r'\n|\ufeff', line)[1]
        else:
            word = re.split(r'\n', line)[0]
        stop_words.append(word)    

In [81]:
trade_war_content_cut = []   #jieba分词
for i in trade_war.content:
    word_cut = jieba.cut(i, cut_all=False)
    word_cut_stop = [word for word in word_cut if word not in stop_words]
    trade_war_content_cut.append(','.join(word_cut_stop))


In [108]:
content0 = trade_war_content_cut[0]  #第0个数据处理
content0_list = re.split(r',', content0)
content0_list[96:100]

['声明', '几点', '感受', '中美']

In [86]:
content0_dict = {}
for word in content0_list:
    if word in content0_dict.keys():
        content0_dict[word] += 1
    else:
        content0_dict[word] = 1

In [88]:
sorted(content0_dict.items(), key= lambda x:x[1], reverse=True )[:20]  #查看前20个词频

[('磋商', 19),
 ('美方', 13),
 ('中美', 9),
 ('中方', 8),
 ('分歧', 7),
 ('公平', 7),
 ('谈', 6),
 ('发展', 6),
 ('中国', 5),
 ('关税', 5),
 ('沟通', 5),
 ('国内', 5),
 ('矛盾', 5),
 ('第十一轮', 4),
 ('经贸', 4),
 ('加征', 4),
 ('消息', 3),
 ('陶然', 3),
 ('笔记', 3),
 ('情况', 3)]

In [90]:
###词向量
tf = TfidfVectorizer(analyzer='word', min_df = 0, stop_words = stop_words) #ngram_range=(1,3)

In [91]:
tfidf_matrix =  tf.fit_transform(trade_war_content_cut)
feature_names = tf.get_feature_names() 
print(len(feature_names))  #词频矩阵的列数

27004


In [92]:
dense = tfidf_matrix.todense()

episode = dense[0].tolist()[0] #这是一个[[1，2，3]]的格式，所以要取[0]
phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0] #将index和结果绑定，并去掉了0的项
sorted_phrase_scores = sorted(phrase_scores, key=lambda x: x[1], reverse=True)  #降序排列
sorted_phrase_scores_20 = [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20] #tuple格式
for phrase, score in sorted_phrase_scores_20:
        print('{0: <20}{1}'.format(phrase, score))  
              

磋商                  0.42988766583253085
美方                  0.20482985081579966
第十一轮                0.20188547419543065
分歧                  0.18877375658623427
公平                  0.16433251565883067
矛盾                  0.14666668748922806
沟通                  0.13790400519000215
笔记                  0.13325352116502898
陶然                  0.13325352116502898
中方                  0.13079922142517067
打打                  0.12490997444127379
谈谈                  0.11509293668348493
平衡点                 0.10596761947682347
中美                  0.10321945794119261
诉求                  0.09862327340641866
两边                  0.09704513552159236
碰撞                  0.09116804558530366
相处                  0.08883568077668598
说服                  0.0867783892138729
另一方                 0.084938079200563


----
## WashingtonPost网址爬取
[WashingtonPost](https://www.washingtonpost.com/)

In [32]:
##不打开浏览器
options = Options() 
options.add_argument("--headless") 


#打开浏览器
driver = webdriver.Chrome()   #options = options
driver.get('https://www.washingtonpost.com/newssearch/?query=China-US%20trade%20war&sort=Relevance&datefilter=12%20Months&startat=0&spellcheck#top')  #打开链接

In [9]:
def url_get_content_WathingtonPost(url):
    """
    该函数根据所传入的字符串类型的链接url获取该页的内容
    示例：url = 'https://baidu.com'
    结果：输出为预处理后的结果
    """
    content = ''
    try:
        html = requests.get(url) #, cookies = cookies, headers = headers) #打开链接  #伪装成真实的访问者
        soup = BeautifulSoup(html.content, "html.parser")  #解码链接
        content = soup.find('article', {"class": "paywall"}).get_text() #解码后找到相应内容
        #content1 = ','.join(re.findall(r'\w+', content0))  #处理初始数据
        
        #试图深度处理数据
#         try:
#             remove = re.findall(r'function.*?fix', content1)[0]  
#             content = content1.replace(remove, '')
#         except:
#             content  = content1
            
    except:
        pass
    
    return content

In [33]:
def Washington_Post_news(pages = 5):

    import time as TM
    
    time_begin = TM.time()  #初始化时间
    col = 0  #查找的第i个内容
    page = 1    #初始化页数
    column_name = ['title', 'url', 'date_time', 'content']   #所获取的内容
    data = pd.DataFrame(columns=column_name)  #初始化结果
    
    print('-----'*8, '\n准备爬取第1页的内容！！')
    os.system('say "准备爬取第1页的内容!"')
    
    while page <= pages:  #页数还没爬完就继续
        
        key = True  #判断是否爬完了某一页的所有内容
        i = 1   #初始化当前页的新闻个数
        
        while key:  #爬取当前页的内容
            
            try:

                url = '//*[@id="main-content"]/div/div/div[2]/div/div[%s]'%(i)   #第i个新闻的位置
                title_url = '/div[1]/p/a'   #第i个新闻的标题位置
                date_time_url = '/div[3]/span'   #第i个新闻的时间信息，


                title = driver.find_element_by_xpath(url+title_url).text  #标题
                href = driver.find_element_by_xpath(url+title_url).get_attribute('href')  #链接
                date_time = driver.find_element_by_xpath(url+date_time_url).text #时间
                content = url_get_content_WathingtonPost(href)  #获得链接中的内容

                #整理数据到data
                data.loc[col, :] = [title, href, date_time, content]

                i += 1  #更新到下一个新闻
                col += 1  #更新col index
                
                ##每10条数据休息一会并告知我
                if col % 10 == 0:
                    print('已经爬取数据%s条！'%(col))  
                    os.system('say "叮叮"')
                    os.system('say "%s"'%(col))
                    TM.sleep(np.random.rand()+3)   #每爬完一页的内容多休息一会
                
                TM.sleep(np.random.rand()+1)  #每爬完一个新闻暂停一会

            except:
                key = False
                
                ##估算剩余时间
                time_cost = TM.time() - time_begin  #当前所用时间
                print('已经用时%.2f秒!!'%(time_cost))
                
                time_mean = time_cost/page  #当前每页的平均时间        
                time_need = time_mean*(pages-page)   #剩余页数的时间
                days,hours,minutes,seconds = time_trans(time_need)  #单位为秒的时间数据处理为日期
                print('预计还需用时\"%d天%02.0f:%02.0f:%02.0f\"!'%(days, hours, minutes, seconds))
                print('-----'*4)
                
                page += 1   #更新到下一页
                print(data.content[len(data)-1])  #查看这一页的最后一个内容
                if page <= pages:
                    print('正在爬取第%s页的内容！！'%(page))
                
                    os.system('say "正在爬取第%s页的内容"'%(page))
                    ##点击下一页
                    driver.find_element_by_link_text('Next').click()   #点击下一页
                
    time_end = TM.time()-time_begin
    days_end,hours_end,minutes_end,seconds_end = time_trans(time_end)
    
    print('用时%d天%02.0f:%02.0f:%02.0f，已经爬完%s页的内容共%s条！！'%( days_end,hours_end,minutes_end,seconds_end, page-1,col))  #已经爬完数据
    print('******'*8)
    
    driver.quit()  #退出并关闭浏览器
    
    os.system('say "WashingtonPost内容已经爬取完成！请查收！"')  #告知我
    
    return data
        
        
        
        
        
        
        

In [34]:
China_US_trade_war = Washington_Post_news(50)

---------------------------------------- 
准备爬取第1页的内容！！
已经爬取数据10条！
已经爬取数据20条！
已经用时239.37秒!!
预计还需用时"0天03:15:29"!
--------------------
正在爬取第2页的内容！！
已经爬取数据30条！
已经爬取数据40条！
已经用时505.27秒!!
预计还需用时"0天03:22:06"!
--------------------
正在爬取第3页的内容！！
已经爬取数据50条！
已经爬取数据60条！
已经用时762.81秒!!
预计还需用时"0天03:19:11"!
--------------------
正在爬取第4页的内容！！
已经爬取数据70条！
已经爬取数据80条！
已经用时1064.62秒!!
预计还需用时"0天03:24:03"!
--------------------
正在爬取第5页的内容！！
已经爬取数据90条！
已经爬取数据100条！
已经用时1370.90秒!!
预计还需用时"0天03:25:38"!
--------------------
正在爬取第6页的内容！！
已经爬取数据110条！
已经爬取数据120条！
已经用时1687.00秒!!
预计还需用时"0天03:26:11"!
--------------------
正在爬取第7页的内容！！
已经爬取数据130条！
已经爬取数据140条！
已经用时1961.90秒!!
预计还需用时"0天03:20:52"!
--------------------
正在爬取第8页的内容！！
已经爬取数据150条！
已经爬取数据160条！
已经用时2255.41秒!!
预计还需用时"0天03:17:21"!
--------------------
正在爬取第9页的内容！！
已经爬取数据170条！
已经爬取数据180条！
已经用时2612.07秒!!
预计还需用时"0天03:18:19"!
--------------------
正在爬取第10页的内容！！
已经爬取数据190条！
已经爬取数据200条！
已经用时2925.66秒!!
预计还需用时"0天03:15:03"!
--------------------
正在爬取第11页的内容！！
已经爬取数据210条！
已经爬取数据220条！
已

In [93]:
China_US_trade_war.head()

,title,url,date_time,content
0,Shares fall in Asia after no deal in China-US ...,https://www.washingtonpost.com/business/shares...,"May 13, 2019",By Elaine Kurtenbach | AP May 13 at ...
1,Shares fall in Asia after no deal in China-US ...,https://www.washingtonpost.com/world/asia_paci...,"May 13, 2019",By Elaine Kurtenbach | AP May 12 at ...
2,Stocks end lower ahead of US-China trade war d...,https://www.washingtonpost.com/business/asian-...,"May 10, 2019",By Stan Choe and Alex Veiga | AP May...
3,Trump administration accuses Chinese officials...,https://www.washingtonpost.com/business/econom...,"May 7, 2019",By David J. Lynch and David J....
4,"Can US, China salvage their talks and end trad...",https://www.washingtonpost.com/business/can-us...,"May 8, 2019",By Paul Wiseman | AP May 7 WASHINGT...


In [94]:
China_US_trade_war.to_csv('China_US_trade_war.csv', index=False)

In [97]:
print(China_US_trade_war.shape)

(954, 4)


In [95]:
df = China_US_trade_war

In [96]:
df.content[948]   #有空值

''

In [54]:
sum(df.content == '')  

171

In [59]:
df_no_none = df.loc[df.content != '', :]  #将非空的数据导出

In [66]:
df_no_none.to_csv('中美贸易战_washington_post.csv', index=False, encoding='utf-8')

In [99]:
df_no_none.shape

(783, 4)

### 数据处理

In [102]:
df_no_none_content = list(df_no_none.content)

In [98]:
tf1 = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,3))

In [104]:
tfidf_matrix1 = tf1.fit_transform(df_no_none_content)

In [106]:
feature_names1 = tf1.get_feature_names() 
print(len(feature_names1))

857436


In [107]:
dense1 = tfidf_matrix1.todense() 

episode1 = dense1[0].tolist()[0]
phrase_scores1 = [pair for pair in zip(range(0, len(episode1)), episode1) if pair[1] > 0]
sorted_phrase_scores1 = sorted(phrase_scores1, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names1[word_id], score) 
    for (word_id, score) in sorted_phrase_scores1][:20]:
        print('{0: <20} {1}'.format(phrase, score))

index                0.09678907164195957
friday               0.09428828190477724
trade talks          0.07174153316291021
fell                 0.06792766406233929
109                  0.06770580920551937
yen                  0.058682233243864006
61                   0.057200328277694466
cents                0.057200328277694466
barrel               0.054698473594984184
talks                0.0489216939428348
crude                0.048316869775101794
asia                 0.048103550699627705
china                0.0478448875401908
tariffs              0.04596624722554272
said                 0.04579810329842932
gained               0.04516004316565522
added                0.04175661518025949
shares               0.04150468079066733
higher               0.041154534448623825
liu                  0.0406140514977835
